In [1]:
%load_ext bigdata
%hive_start
%timeout 300

In [2]:
!hdfs dfs -rm -r -f /tmp/output

Deleted /tmp/output


In [3]:
!hdfs dfs -mkdir /tmp/output

In [4]:
!hdfs dfs -copyFromLocal data.tsv /tmp/output/

In [5]:
%%hive
DROP TABLE IF EXISTS t0;
CREATE TABLE t0 (
    c1 STRING,
    c2 ARRAY<CHAR(1)>, 
    c3 MAP<STRING, INT>
    )
    ROW FORMAT DELIMITED 
        FIELDS TERMINATED BY '\t'
        COLLECTION ITEMS TERMINATED BY ','
        MAP KEYS TERMINATED BY '#'
        LINES TERMINATED BY '\n';
LOAD DATA LOCAL INPATH 'data.tsv' INTO TABLE t0;

DROP TABLE IF EXISTS t0;
OK
Time taken: 14.37 seconds
CREATE TABLE t0 (
    c1 STRING,
    c2 ARRAY<CHAR(1)>, 
    c3 MAP<STRING, INT>
    )
    ROW FORMAT DELIMITED 
        FIELDS TERMINATED BY '\t'
        COLLECTION ITEMS TERMINATED BY ','
        MAP KEYS TERMINATED BY '#'
        LINES TERMINATED BY '\n';
OK
Time taken: 4.247 seconds
LOAD DATA LOCAL INPATH 'data.tsv' INTO TABLE t0;
Loading data to table default.t0
OK
Time taken: 1.162 seconds


In [10]:
%%hive

DROP TABLE IF EXISTS resultado;

CREATE TABLE resultado
AS
SELECT  letra,
        key,
        COUNT(letra)
FROM(
    SELECT letra,
            key
    FROM 
    t0 LATERAL VIEW explode (c2) t0 AS letra
    LATERAL VIEW explode (c3) t0 AS key, val
) t 

GROUP BY
    letra, 
    key
;

INSERT OVERWRITE DIRECTORY '/tmp/output'
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
SELECT * FROM resultado;

DROP TABLE IF EXISTS resultado;
OK
Time taken: 0.011 seconds
CREATE TABLE resultado
AS
SELECT  letra,
        key,
        COUNT(letra)
FROM(
    SELECT letra,
            key
    FROM 
    t0 LATERAL VIEW explode (c2) t0 AS letra
    LATERAL VIEW explode (c3) t0 AS key, val
) t 
GROUP BY
    letra, 
    key
;
Query ID = root_20200119014413_5987cddf-39dd-4f9c-8d50-3f69986848a2
Total jobs = 1
Launching Job 1 out of 1
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1579385130637_0026, Tracking URL = http://282e0d2d9a5d:8088/proxy/application_1579385130637_0026/
Kill Command = /usr/local/hadoop/bin/hadoop job  -kill job_1579385130637_0026
Hadoop job information for 

In [13]:
!hdfs dfs -copyToLocal /tmp/output output;

In [14]:
%%hive
DROP TABLE resultado;

DROP TABLE resultado;
OK
Time taken: 0.115 seconds


In [15]:
%hive_quit